<a href="https://colab.research.google.com/github/ganeshmp01/DS-Classwork/blob/main/First_Impressions_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ganeshmp01","key":"ad99cb76078e0a4514cad9af0d045e21"}'}

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d tapakah68/impressive-dataset

Dataset URL: https://www.kaggle.com/datasets/tapakah68/impressive-dataset
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 99% 5.21G/5.24G [01:10<00:00, 86.9MB/s]
100% 5.24G/5.24G [01:10<00:00, 79.6MB/s]


In [11]:
!unzip -q /content/impressive-dataset.zip

replace First Impression Dataset/first_impression_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [12]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchvision import models, transforms
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Load dataset
data = pd.read_csv('/content/First Impression Dataset/first_impression_dataset.csv')



In [39]:
# prompt:  use the image files from this path /content/First Impression Dataset/images

import numpy as np
import cv2
from PIL import Image

image_path = '/content/First Impression Dataset/images'

# Example: Load and process the first image
image_files = os.listdir(image_path)
if image_files:
  first_image_file = os.path.join(image_path, image_files[0])
  try:
    img = Image.open(first_image_file)
    # You can perform further processing on the image here, e.g., resizing, converting to numpy array
    # img_array = np.array(img)
    print(f"Loaded image: {first_image_file}")
  except Exception as e:
    print(f"Error loading image: {e}")
else:
  print("No image files found in the directory.")


Loaded image: /content/First Impression Dataset/images/15738114470390050.jpg


In [58]:
# Preprocessing
class ImpressionsDataset(Dataset):
    def __init__(self, texts, images, tokenizer, transform=None):
        self.texts = texts
        self.images = images
        self.tokenizer = tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.texts)


    def __getitem__(self, idx):
        text = self.texts[idx]
        image_id = self.images[idx]

        # Process text
        if isinstance(text, str):
            inputs = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
        else:
            # Handle missing or non-string values (e.g., replace with empty string)
            inputs = self.tokenizer('', return_tensors='pt', padding='max_length', truncation=True, max_length=128)

        # Process image
        from PIL import Image
        image_path = f'/content/First Impression Dataset/images/{image_id}.jpg'
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return inputs, image

# Set up tokenization and transformations
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create dataset and dataloaders
train_texts, test_texts, train_images, test_images = train_test_split(
    data['impression_en'].tolist(), data['image_id'].tolist(), test_size=0.2, random_state=42)

train_dataset = ImpressionsDataset(train_texts, train_images, tokenizer, transform)
test_dataset = ImpressionsDataset(test_texts, test_images,  tokenizer, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the multimodal model
class MultimodalModel(nn.Module):
    def __init__(self):
        super(MultimodalModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, 256)

        self.fc = nn.Linear(256 + 768, 1)  # Combine image and text outputs

    def forward(self, text_inputs, images):
        text_outputs = self.bert(**text_inputs).pooler_output
        image_outputs = self.cnn(images)

        combined = torch.cat((text_outputs, image_outputs), dim=1)
        return self.fc(combined)



In [59]:
%%time
# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultimodalModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(5):  # Adjust epochs as needed
    model.train()
    running_loss = 0.0
    for text_inputs, images in train_loader:
        optimizer.zero_grad()

        text_inputs = {key: val.squeeze(1).to(device) for key, val in text_inputs.items()}
        images = images.to(device)

        # Assuming your labels are available as a tensor named 'labels'
        labels = torch.randint(0, 2, (images.size(0), 1)).float().to(device) # Example random labels - replace with your actual labels

        # Forward pass
        outputs = model(text_inputs, images)

        # Calculate loss with outputs and labels
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch + 1}, Loss: {avg_loss}')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.6956153619289398
Epoch 2, Loss: 0.6959856454133987
Epoch 3, Loss: 0.6958034926652908
Epoch 4, Loss: 0.6953596341609954
Epoch 5, Loss: 0.694644413471222
CPU times: user 50min 36s, sys: 3min 20s, total: 53min 56s
Wall time: 54min 28s


In [60]:
from sklearn.metrics import accuracy_score, f1_score

In [62]:
    # Evaluation
    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for text_inputs, images in test_loader:
            text_inputs = {key: val.squeeze(1).to(device) for key, val in text_inputs.items()}
            images = images.to(device)

            outputs = model(text_inputs, images)
            preds = torch.sigmoid(outputs).squeeze().cpu().numpy()
            val_preds.extend(preds)

            # Extract labels for current batch
            batch_indices = range(len(text_inputs['input_ids']))
            batch_labels = torch.tensor([labels[i] for i in batch_indices], dtype=torch.float).cpu().numpy()
            val_labels.extend(batch_labels)

    # Convert predictions to binary
    val_preds = np.array(val_preds) > 0.5
    val_labels = np.array(val_labels)

    # Calculate accuracy and F1-score
    accuracy = accuracy_score(val_labels, val_preds)
    f1 = f1_score(val_labels, val_preds)

    print(f'Epoch {epoch + 1} Validation Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}')

Epoch 5 Validation Accuracy: 0.4913, F1 Score: 0.5231


In [64]:
# Prediction step
def predict(texts, images, model, tokenizer, transform):
    model.eval()
    preds = []
    with torch.no_grad():
        for i in range(len(texts)):
            text = texts[i]
            image_file = images[i]

            # Process text
            inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
            inputs = {key: val.squeeze(1).to(device) for key, val in inputs.items()}

            # Process image
            image = Image.open(image_file).convert('RGB')
            image = transform(image).unsqueeze(0).to(device)

            # Forward pass
            output = model(inputs, image)
            pred = torch.sigmoid(output).item()
            preds.append(pred)

    return preds

# Example usage of the predict function
texts_to_predict = ["A young beautiful girl from the East. The natural hair color is dark, in this case with highlights. Lovely smile, beautiful dark eyes. Approximate age from twenty to twenty-seven years old. Presumably a student of the humanities faculty.",
                    "A young man in a hood on Red Square, around 30 years old. Nice, kind, a little sad and tired. Most likely works as a programmer. A responsive person with an open heart. It's pleasant to talk to someone like that."]
images_to_predict = ["/content/First Impression Dataset/images/15708024981269913.jpg", "/content/First Impression Dataset/images/15708029957387770.jpg"]

predictions = predict(texts_to_predict, images_to_predict, model, tokenizer, transform)
print("Predictions:", predictions)

Predictions: [0.5134896636009216, 0.5128973126411438]
